In [1]:
import os
import sys
import numpy as np
import pandas as pd
import random
import logging
from datetime import date, datetime, timedelta
from dateutil import parser
import pandas_profiling
import matplotlib.pyplot as plt
import warnings
from ml_utils import Pipeline

warnings.filterwarnings('ignore')

# Homework 3 - Improving the Pipeline
### Justin Cohler

For this assignment, I have built out a version the Pipeline located in ml_utils (same directory) with functions for looping through parameter grids for several models. 

I included Logistic Regression, K-Nearest Neighbors, Decision Trees, Random Forests, Gradient Boosting, and Bagging in the runs below. I configured SVMs for running, but was unable to run in a timely fashion on the full dataset, so it is not included in the result set below. 

Additionally, I ran temporal validation on 2-month intervals from 6/1/2013 to 12/31/2013, always testing on a 2-month interval with a training window that would grow backwards in time. This was more reasonable than running over 3 years for time-to-compute. The results dataframe is shown below. Precision-Recall graphs are all in this same directory, named by their model and run numbers. 

## The winning model
The best model found was a Random Forest Classifier, with the following properties:

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

And the following result metrics:

- auc-roc:                                                0.717098
- p_at_1:                                                 0.977011
- p_at_2:                                               0.961686
- p_at_5:                                                0.961686
- p_at_10:                                                0.954406
- p_at_20:                                               0.915342
- p_at_30:                                                0.87883
- p_at_50:                                               0.815995
- r_at_1:                                              0.0145457
- r_at_2:                                                 0.028635
- r_at_5:                                                0.0715875
- r_at_10:                                                0.142091
- r_at_20:                                                0.272603
- r_at_30:                                                0.392619
- r_at_50:                                                0.607609

The Precision-Recall graph for the optimal RF model is below (all other precision-recall graphs are in this same directory):
!['Precision Recall - Random Forest (Gini, Max Depth of 50)'](RF1.png)


# Parameter Grid
Below, the following code prints out the parameter grid for the models, which is housed in the ml_utils Pipeline class. 

The pipeline.generate_classifiers() method prints out a dictionary of SKLearn models, as well as their respective parameter grids.

In [18]:
logger = logging.getLogger('hw3')

pd.options.display.max_rows = 999

pipeline = Pipeline()
pipeline.generate_classifiers()
pipeline.classifiers

{'BAG': {'params': {'bootstrap_features': [False, True],
   'max_features': [5, 20],
   'max_samples': [5],
   'n_estimators': [10]},
  'type': BaggingClassifier(base_estimator=None, bootstrap=True,
           bootstrap_features=False, max_features=1.0, max_samples=1.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)},
 'DT': {'params': {'criterion': ['gini', 'entropy'],
   'max_depth': [5, 50],
   'min_samples_split': [2, 10]},
  'type': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, presort=False, random_state=None,
              splitter='best')},
 'GB': {'params': {'learning_rate': [0.5],
   'max_depth': [1, 5],
   'n_estimators': [5, 10],
   'subsample': [0.5]},
  'type': 

## Data Exploration

We can draw the following information from the profile report analyzed below:
- Most of the projects are from High and Highest Poverty classrooms/schools
- Many of the projects are focused on the subject of Literacy (28,050/66,562), nearly half!
- Books, supplies, and technology comprise the majority of the resources teachers request.
- Cities and counties are fairly well distributed, showing that Donors Choose has been popularized as a platform across the country.
- Most of the projects originate from schools in urban areas (44,452/50,180) 
- The vast majority of teachers requesting projects are women (Ms and Mrs. prefixes comprise much more than 50% of the rows, closer to 90%)
- Pearson and Spearman correlation matrices look similar, implying that outliers aren't a major issue in the data.


In [8]:
pandas_profiling.ProfileReport(projects)

Number of variables,116
Number of observations,94612
Total Missing (%),0.0%
Total size in memory,23.9 MiB
Average record size in memory,265.0 B
Numeric,12
Categorical,0
Boolean,97
Date,1
Text (Unique),1
Rejected,5


In [4]:
start = parser.parse("2013-06-01")
end = parser.parse("2013-12-31")

logger.info("Ingesting dataframes...")
outcomes = pipeline.ingest('data/outcomes.csv')
projects = pipeline.ingest('data/projects.csv')


Ingesting dataframes...


## Model Running
The run_temporal method runs Logistic Regression, KNearest Neighbors, Decision Trees, Random Forest, Gradient Boosting, and Bagging. 

### Baselining
Additionally, run_temporal generates a random baseline, denoted by "RAND". The baseline takes the proportion of fully funded projects in the given training set and creates a proportionally random test set from the training set's percentage. If 30% of projects in the train set were funded, the test set will choose 0.3 as the threshold for N random floats between 0 and 1, where N is the length of the test set. It is then tested against the true values like the actual models, and is added to the results DataFrame.

In [5]:

outcomes.set_index("projectid")
outcomes = outcomes.drop(['is_exciting', 'at_least_1_teacher_referred_donor',  'at_least_1_green_donation', 'great_chat', 'three_or_more_non_teacher_referred_donors',
                          'one_non_teacher_referred_donor_giving_100_plus', 'donation_from_thoughtful_donor', 'great_messages_proportion', 'teacher_referred_count', 'non_teacher_referred_count'], axis=1)

projects["date_posted"] = pd.to_datetime(projects["date_posted"])
projects.set_index("date_posted")
projects = projects[projects['date_posted'].between(
    start, end, inclusive=True)]

projects = pipeline.presplit(projects)
logger.info("Project dimensions after feature generation: {}".format(projects.shape))

baselines = ['RAND']
results_df = pipeline.run_temporal(['LR', 'KNN', 'DT', 'RF', 'GB', 'BAG'], projects, outcomes, start, end, baselines)

Generating features...
Generated dummy features.
Generated labelized features.
Made binaries from 't' 'f' pair columns.
Finished feature generation.
Project dimensions after feature generation: (94612, 115)
Temporally validating on:
Train: 2013-08-30 00:00:00 - 2013-10-30 00:00:00
Test: 2013-10-31 00:00:00 - 2013-12-31 00:00:00
Prediction window: 2 months
Merging dataframes...
DataFrames merged with dimensions: train(62043, 116), test(26109, 116).
Splitting X and y, train and test...
Running LR...
Running with params {'C': 0.01, 'penalty': 'l1'}
Running with params {'C': 0.01, 'penalty': 'l2'}
Running with params {'C': 0.1, 'penalty': 'l1'}
Running with params {'C': 0.1, 'penalty': 'l2'}
LR finished.
Running KNN...
Running with params {'algorithm': 'ball_tree', 'n_neighbors': 10, 'weights': 'uniform'}
Running with params {'algorithm': 'ball_tree', 'n_neighbors': 10, 'weights': 'distance'}
Running with params {'algorithm': 'ball_tree', 'n_neighbors': 20, 'weights': 'uniform'}
Running wi

DT finished.
Running RF...
Running with params {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 10}
Running with params {'max_depth': 50, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 10}
RF finished.
Running GB...
Running with params {'learning_rate': 0.5, 'max_depth': 1, 'n_estimators': 5, 'subsample': 0.5}
Running with params {'learning_rate': 0.5, 'max_depth': 1, 'n_estimators': 10, 'subsample': 0.5}
Running with params {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 5, 'subsample': 0.5}
Running with params {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 10, 'subsample': 0.5}
GB finished.
Running BAG...
Running with params {'bootstrap_features': False, 'max_features': 5, 'max_samples': 5, 'n_estimators': 10}
Running with params {'bootstrap_features': False, 'max_features': 20, 'max_samples': 5, 'n_estimators': 10}
Running with params {'bootstrap_features': True, 'max_features': 5, 'max_samples': 5, 'n_estimators': 10

## Results
Below, the results are sorted by the highest auc-roc score.

In [17]:
results_df.sort_values(by=['auc-roc'], ascending=False)

,model_type,clf,parameters,auc-roc,p_at_1,p_at_2,p_at_5,p_at_10,p_at_20,p_at_30,p_at_50,r_at_1,r_at_2,r_at_5,r_at_10,r_at_20,r_at_30,r_at_50
51,RF,"(DecisionTreeClassifier(class_weight=None, cri...","{'max_depth': 5, 'max_features': 'sqrt', 'min_...",0.717098,0.977011,0.961686,0.961686,0.954406,0.915342,0.878830,0.815995,0.014546,0.028635,0.071587,0.142091,0.272603,0.392619,0.607609
23,GB,([DecisionTreeRegressor(criterion='friedman_ms...,"{'learning_rate': 0.5, 'max_depth': 1, 'n_esti...",0.710538,0.973180,0.971264,0.951724,0.938697,0.906914,0.877298,0.814999,0.014489,0.028920,0.070846,0.139752,0.270093,0.391934,0.606868
55,GB,([DecisionTreeRegressor(criterion='friedman_ms...,"{'learning_rate': 0.5, 'max_depth': 5, 'n_esti...",0.706727,0.957854,0.971264,0.963985,0.949042,0.913810,0.875511,0.806879,0.014260,0.028920,0.071759,0.141293,0.272146,0.391136,0.600821
56,GB,([DecisionTreeRegressor(criterion='friedman_ms...,"{'learning_rate': 0.5, 'max_depth': 5, 'n_esti...",0.703580,0.973180,0.969349,0.960920,0.937548,0.903275,0.868105,0.810556,0.014489,0.028863,0.071530,0.139581,0.269009,0.387827,0.603559
20,RF,"(DecisionTreeClassifier(class_weight=None, cri...","{'max_depth': 5, 'max_features': 'sqrt', 'min_...",0.702334,0.961686,0.959770,0.957088,0.941379,0.905382,0.868488,0.804887,0.014317,0.028578,0.071245,0.140152,0.269637,0.387998,0.599338
54,GB,([DecisionTreeRegressor(criterion='friedman_ms...,"{'learning_rate': 0.5, 'max_depth': 1, 'n_esti...",0.700499,0.977011,0.936782,0.955556,0.928736,0.892166,0.861593,0.820898,0.014546,0.027893,0.071131,0.138269,0.265701,0.384918,0.611260
24,GB,([DecisionTreeRegressor(criterion='friedman_ms...,"{'learning_rate': 0.5, 'max_depth': 5, 'n_esti...",0.700103,0.973180,0.957854,0.951724,0.943295,0.907489,0.867722,0.810709,0.014489,0.028521,0.070846,0.140437,0.270264,0.387656,0.603673
25,GB,([DecisionTreeRegressor(criterion='friedman_ms...,"{'learning_rate': 0.5, 'max_depth': 5, 'n_esti...",0.692787,0.965517,0.959770,0.948659,0.943295,0.906723,0.870787,0.804581,0.014375,0.028578,0.070618,0.140437,0.270036,0.389025,0.599110
47,DT,"DecisionTreeClassifier(class_weight=None, crit...","{'criterion': 'entropy', 'max_depth': 5, 'min_...",0.691617,0.904215,0.948276,0.944828,0.926820,0.909979,0.832482,0.835146,0.013462,0.028236,0.070333,0.137984,0.271006,0.371913,0.621870
48,DT,"DecisionTreeClassifier(class_weight=None, crit...","{'criterion': 'entropy', 'max_depth': 5, 'min_...",0.691617,0.904215,0.948276,0.944828,0.926820,0.909979,0.832482,0.835146,0.013462,0.028236,0.070333,0.137984,0.271006,0.371913,0.621870
